In [5]:
# Import the pandas library for data manipulation and analysis
import pandas as pd

# Load the expense data from the CSV file into a pandas DataFrame
df = pd.read_csv("data/expenses_sample.csv")

# Display the first five rows of the dataset
# This is used to quickly verify that the data has been loaded correctly
df.head()

,date,description,category,amount
0,2025-01-02,Coffee,Food,3.50
1,2025-01-03,Bus ticket,Transport,2.75
2,2025-01-04,Groceries,Food,45.20
3,2025-01-06,Lunch,Food,9.80
4,2025-01-07,Gas,Transport,40.00
